# Self-Driving Cab
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [2]:
# !pip install cmake gym[atari] scipy
# !pip install gym[toy_text]
# !pip install stable_baselines3

In [1]:
import numpy as np
import gym
import random
from tabulate import tabulate
from IPython.display import clear_output
from time import sleep

### Gym's interface

In [2]:
# https://stackoverflow.com/questions/57263759/how-can-i-start-the-environment-from-a-custom-initial-state-for-mountain-car
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()  # reset environment to a new, random state
env = env.unwrapped  # to access the inner functionalities of the class
print(env.render())

print(
    "Action Space {}".format(env.action_space)
)  # south, north, east, west, pickup, dropoff = 6
print("State Space {}".format(env.observation_space))  # 5x5x(4+1)x4=500
# 5x5 board size
# 4+1 - number of states for passengers, as the passenger may already be in the taxi
# 4 - number of dropoff states


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)


In [3]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
print(env.render())

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




### The Reward Table

In [4]:
env.P[328] # {action: [(probability, nextstate, reward, done)]}
# - The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.
# - In this env, probability is always 1.0.
# - The nextstate is the state we would be in if we take the action at this index of the dict
# - All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi
# has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
# done is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode.
# - he source code has made it impossible to actually move the taxi across a wall, so if the taxi chooses that action,
# it will just keep accruing -1 penalties, which affects the long-term reward.

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Solving the environment without Reinforcement Learning

In [5]:
env.s = 328  # set environment to illustration's state

epochs, penalties, reward = 0, 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample() # this method automatically selects one random action from set of all possible actions.
    state, reward, done, info, _ = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 946
Penalties incurred: 282


### Rendering solution

In [6]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (West)

Timestep: 100
State: 376
Action: 3
Reward: -1


### Enter Reinforcement Learning

#### Implementing Q-learning in python

In [ ]:
# Initialize the Q-table by all zeros.
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(tabulate(q_table, tablefmt="psql"))
print(q_table.shape)

+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 

In [ ]:
%%time
"""Training the agent"""

# Hyperparameters
alpha   = 0.1
gamma   = 0.6 # high value for the discount factor (close to 1) captures the long-term effective award (gamma -> 0 => greedy agent)
epsilon = 0.1 # lower epsilon value results in episodes with more penalties

# For plotting metrics
all_epochs = []
all_penalties = []

episodes = 100000

for i in range(1, episodes+1):
    state = env.reset()[0]

    epoch, penalties, reward = 0, 0, 0
    done = False

    # state == y (rows of q_table), action == x (columns of q_table)
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space (integer in [0, 5])
        else:
            action = np.argmax(q_table[state]) # Exploit learned values (position of biggest q_value from q_table)

        next_state, reward, done, info, _ = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state]) # maxmium possible q_value to get

        # equation for Q-learning
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: total: 1min 7s
Wall time: 1min 2s


In [ ]:
print(tabulate(q_table, tablefmt="psql"))

+------------+------------+------------+-----------+-----------+-----------+
|  0         |  0         |  0         |  0        |   0       |   0       |
| -2.41837   | -2.36395   | -2.41837   | -2.36395  |  -2.27325 | -11.364   |
| -1.87014   | -1.45024   | -1.87014   | -1.45024  |  -0.7504  | -10.4502  |
| -2.36395   | -2.27325   | -2.36395   | -2.27325  |  -2.12209 | -11.2733  |
| -2.49619   | -2.49667   | -2.49619   | -2.49649  |  -9.82328 |  -8.99909 |
|  0         |  0         |  0         |  0        |   0       |   0       |
| -2.49619   | -2.49686   | -2.49619   | -2.49659  | -10.7917  | -10.022   |
| -2.48237   | -2.48582   | -2.48237   | -2.48619  |  -9.01778 |  -8.11468 |
| -2.27325   | -2.35063   | -2.37279   | -2.322    |  -8.96829 |  -6.70995 |
| -2.47061   | -2.47934   | -2.48059   | -2.47679  |  -9.67764 | -10.3132  |
|  0         |  0         |  0         |  0        |   0       |   0       |
| -2.45102   | -2.46272   | -2.46265   | -2.46591  | -10.4117  |  -9.25075 |

In [ ]:
print(q_table[328])
print(np.argmax(q_table[328]))
# the max Q-value is "north" - index 1

[ -2.38032143  -2.27325184  -2.40381073  -2.35964455 -10.9031543
 -10.31714325]
1


### Evaluating the agent

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()[0]
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info, _ = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.26
Average penalties per episode: 0.0


### Visualization

In [ ]:
init_state = 328
env.s = init_state  # set environment to illustration's state

epochs, penalties, reward = 0, 0, 0

frames = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info, _ = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    
    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 12
Penalties incurred: 0


In [ ]:
env.s = init_state
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 12
State: 0
Action: 5
Reward: 20
